In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yp-toxic-comments/toxic_comments.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch_xla.core.xla_model as xm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score
from tqdm import tqdm

ModuleNotFoundError: No module named 'torch_xla'

# Загрузка и предобработка данных

In [ ]:
# Загрузка датасета
data = pd.read_csv("/kaggle/input/yp-toxic-comments/toxic_comments.csv", index_col=0)

data.info()
data.head()
print("Проверка на пропуски:\n", data.isnull().sum())

In [ ]:
print(data['toxic'].value_counts(normalize=True))

In [ ]:
# Разделение данных на обучающий и тестовый наборы (80% train, 20% test)
train_data, test_data = train_test_split(
    data,
    test_size=0.2,
    random_state=42,
    stratify=data['toxic']  # Сохранение пропорций классов
)

In [ ]:
# Токенизация текста
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

def tokenize_data(data):
    return tokenizer(
        list(data['text']),
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

train_encodings = tokenize_data(train_data)
test_encodings = tokenize_data(test_data)

In [ ]:
# Создание датасетов
def create_dataset(encodings, labels):
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']
    labels = torch.tensor(labels.values)
    return TensorDataset(input_ids, attention_mask, labels)

train_dataset = create_dataset(train_encodings, train_data['toxic'])
test_dataset = create_dataset(test_encodings, test_data['toxic'])

# Создание DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Загрузка модели на TPU
device = xm.xla_device()
model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
model.to(device)

# Оценка модели
def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Оценка"):
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return f1_score(all_labels, all_preds, average='weighted')

f1_test = evaluate(model, test_loader)
print(f'F1 на тестовом наборе: {f1_test:.4f}')